<a href="https://colab.research.google.com/github/mnaseri94/Sleep-Wake-DL/blob/main/LSTM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **GPU**

In [ ]:
!nvidia-smi

# **install**

In [ ]:
!pip install torchmetrics

# **Imports 📢**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split

import torchvision
from torchvision import transforms as T

from torchmetrics import Accuracy
from tqdm import tqdm

# **Dataset 🗂️**

In [ ]:
cd /content/drive/MyDrive/Deep-learning-howsam/RNN

In [ ]:
data = pd.read_csv('train-2.csv' ,  header=None , sep=',')

In [ ]:
data = data.rename(columns={0:'date' , 1:'time' , 2:'sensor' , 3:'key' , 4:'activity' , 5:'start_end_activity'})

In [ ]:
data.head()

### **Unique Values of 'start_end_activity' feature**

In [ ]:
data['start_end_activity'].unique()

### **fill NaN Values with ffill method**

In [ ]:
data.isnull().sum()

In [ ]:
data['activity'].fillna(method='ffill' , inplace=True)

In [ ]:
data['start_end_activity'].fillna(method='ffill' , inplace=True)

### **Create another class to represent other activities ✔**

In [ ]:
flags = data['start_end_activity'] == 'end'

In [ ]:
data['activity'][flags] = 0

In [ ]:
indices = []
activitis = []

for i in range(1 , len(data['activity'])):

    if data['activity'][i] == 0 and data['activity'][i-1] != 0:

        indices.append(i)
        activitis.append(data['activity'][i-1])

In [ ]:
for idx , act in zip(indices,activitis):
    data['activity'][idx] = act

In [ ]:
# Now we dont need this feature enymore
data.drop('start_end_activity' , axis=1 , inplace=True)

In [ ]:
# we want replace int value 0 to str 0
flags = data['activity'] == 0

In [ ]:
data['activity'][flags] = '0'

### **Now Save Dataset 📚**

In [ ]:
data.to_csv('dataset.csv' , index=False)

### **Check Missing Values ✅**

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(10 , 4))
sns.barplot(data = data ,
            x=data['activity'].value_counts().index ,
            y=data['activity'].value_counts().values,
            );
plt.xticks(rotation = 90);

### **Some Preprocess and Visualization on Data 📊**

In [ ]:
data2 = data.copy()

In [ ]:
le = LabelEncoder()

In [ ]:
data2['activity'] = le.fit_transform(data2['activity'])

In [ ]:
plt.figure(figsize=(10 , 4))
sns.barplot(data = data2 ,
            x=data2['activity'].value_counts().index ,
            y=data2['activity'].value_counts().values,
            );
plt.xticks(rotation = 90);

In [ ]:
le.classes_

### **Split data based on Sensors**

In [ ]:
df_Motion = data2.loc[data2['sensor'].str.startswith('M')]

In [ ]:
df_Motion.shape

In [ ]:
plt.figure(figsize=(12 , 4));

df_Motion_one_day = df_Motion[df_Motion['date'] == '2009-10-16']
plt.plot(df_Motion_one_day['time'] , df_Motion_one_day['sensor'] , color='blue');
plt.xticks([]);
plt.title('2009-10-16');

In [ ]:
plt.figure(figsize=(12 , 4));

df_Motion_one_day = df_Motion[df_Motion['date'] == '2009-10-17']
plt.plot(df_Motion_one_day['time'] , df_Motion_one_day['sensor'] , color='red');
plt.xticks([]);
plt.title('2009-10-17');

In [ ]:
plt.figure(figsize=(12 , 4));

df_Motion_one_day = df_Motion[df_Motion['date'] == '2009-10-18']
plt.plot(df_Motion_one_day['time'] , df_Motion_one_day['sensor'] , color='green');
plt.xticks([]);
plt.title('2009-10-18');

In [ ]:
df_temp = data2.loc[data2['sensor'].str.startswith('T')]

In [ ]:
df_temp.shape

In [ ]:
plt.figure(figsize=(12 , 4));

df_temp_one_day1 = df_temp[df_temp['date'] == '2009-10-16']
df_temp_one_day2 = df_temp[df_temp['date'] == '2009-10-17']
df_temp_one_day3 = df_temp[df_temp['date'] == '2009-10-18']

plt.plot(df_temp_one_day1['time'] , df_temp_one_day1['key'] , color='blue' , label='2009-10-16');
plt.plot(df_temp_one_day2['time'] , df_temp_one_day2['key'] , color='red' , label='2009-10-17');
plt.plot(df_temp_one_day3['time'] , df_temp_one_day3['key'] , color='green' , label='2009-10-18');
plt.legend()
plt.xticks([]);

In [ ]:
df_door = data2.loc[data2['sensor'].str.startswith('D')]

In [ ]:
df_door.shape

In [ ]:
plt.figure(figsize=(12 , 4));

df_temp_one_day1 = df_door[df_door['date'] == '2009-10-16']
df_temp_one_day2 = df_door[df_door['date'] == '2009-10-17']
df_temp_one_day3 = df_door[df_door['date'] == '2009-10-18']

plt.plot(df_temp_one_day1['time'] , df_temp_one_day1['sensor'] , color='blue' , label='2009-10-16');
plt.plot(df_temp_one_day2['time'] , df_temp_one_day2['sensor'] , color='red' , label='2009-10-17');
plt.plot(df_temp_one_day3['time'] , df_temp_one_day3['sensor'] , color='green' , label='2009-10-18');
plt.legend()
plt.xticks([]);

### **Encode Features**

In [ ]:
unvalid_keys = ['ON`','ON0','O']
indices = []
for key in unvalid_keys:
    idx = df_Motion[df_Motion['key'] == key].index.item()
    indices.append(idx)

In [ ]:
df_Motion['key'][indices] = 'ON'

In [ ]:
le_key = LabelEncoder()
df_Motion['key'] = le_key.fit_transform(df_Motion['key'])

In [ ]:
le_sensor = LabelEncoder()
df_Motion['sensor'] = le_sensor.fit_transform(df_Motion['sensor'])

In [ ]:
le_sensor.classes_

In [ ]:
le_key.classes_

# **Slicing Window**

In [ ]:
time_step = 80
offset = 50
features = ['sensor', 'key']
target = 'activity'

X = []
y = []
for i in range(0 , len(df_Motion)-time_step , offset):
    X.append(df_Motion.iloc[i:i+time_step][features])
    y.append(df_Motion.iloc[i+time_step][target])
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

# **Split data to train and valid ⚡**

In [ ]:
X_train = X[:-1000]
X_valid = X[-1000:]
y_train = y[:-1000]
y_valid = y[-1000:]

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

# **TensorDataset**

In [ ]:
train_set = TensorDataset(X_train, y_train)
test_set = TensorDataset(X_valid, y_valid)

# **DataLoader**

In [ ]:
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(test_set, batch_size=50, shuffle=False)

In [ ]:
x_batch, y_batch = next(iter(train_loader))
print(x_batch.shape)
print(y_batch.shape)

# **Model 🧠**

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, RNN, input_size, hidden_size, num_layers, bidirectional, num_cls):
        super().__init__()
        self.rnn = RNN(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          bidirectional=bidirectional,
                          batch_first=True)

        self.dropout = nn.Dropout(p=0.3)

        self.fc = nn.LazyLinear(num_cls)

    def forward(self, x):
        outputs, _ = self.rnn(x)
        y = self.fc(outputs[:, -1, :]) # out: many[:, -1, :]
        y = self.dropout(y)
        return y

In [ ]:
model = RNNModel(nn.LSTM, 2, 80, 3, False, 16)
model

In [ ]:
model(x_batch).shape

### **Params**

In [ ]:
def num_params(model, k=1e6):
    nums = sum(p.numel() for p in model.parameters())/k
    return nums

In [ ]:
num_params(model, 1e3)

# **Device ⚙️**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# **Utils 🧰**

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# **Functions** 🧮

In [ ]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
    model.train()
    loss_train = AverageMeter()
    acc_train = Accuracy(task='multiclass', num_classes=16).to(device)
    with tqdm(train_loader, unit="batch") as tepoch:
        for inputs, targets in tepoch:
            if epoch is not None:
                tepoch.set_description(f"Epoch {epoch}")
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            loss = loss_fn(outputs, targets)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train.update(loss.item())
            acc_train(outputs, targets.int())
            tepoch.set_postfix(loss=loss_train.avg,
                         accuracy=100.*acc_train.compute().item())
    return model, loss_train.avg, acc_train.compute().item()

In [ ]:
def validation(model, test_loader, loss_fn):
    model.eval()
    with torch.no_grad():
        loss_valid = AverageMeter()
        acc_valid = Accuracy(task='multiclass', num_classes=16).to(device)
        for i, (inputs, targets) in enumerate(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            loss_valid.update(loss.item())
            acc_valid(outputs, targets.int())
    return loss_valid.avg, acc_valid.compute().item()

# **Efficient way for set hyperparams 🔨**

### Step 1: check forward path

Calculate loss for one batch

In [ ]:
model = RNNModel(nn.LSTM, 2, 80, 3, False, 16)
model.to(device)
loss_fn = nn.CrossEntropyLoss()

x_batch, y_batch = next(iter(train_loader))
outputs = model(x_batch.to(device))
loss = loss_fn(outputs, y_batch.to(device))
print(loss)

### Step 2: check backward path

Select 5 random batches and train the model

In [ ]:
_, mini_train_dataset = random_split(train_set, (len(train_set)-500, 500))
mini_train_loader = DataLoader(mini_train_dataset, 20)

In [ ]:
model = RNNModel(nn.LSTM, 2, 80, 3, False, 16)
model.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    model, _, _ = train_one_epoch(model, mini_train_loader, loss_fn , optimizer, epoch)

### Step 3: select best lr

Train all data for one epoch

In [ ]:
num_epochs = 5
for lr in [0.2, 0.1, 0.01 , 0.001 , 0.0001]:
  print(f'LR={lr}')
  model = RNNModel(nn.LSTM, 2, 80, 3, False, 16).to(device)
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
  for epoch in range(num_epochs):
    model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
  print()

### Step 4: small grid (optional)

Create a small grid based on the WD and the best LR

In [ ]:
num_epochs = 5

for lr in [0.0001, 0.0005, 0.001, 0.0015, 0.002]:
  for wd in [1e-4, 1e-5, 0.]:
    model = RNNModel(nn.LSTM, 2, 80, 3, False, 16).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    print(f'LR={lr}, WD={wd}')

    for epoch in range(num_epochs):
      model, loss, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
    print()

### Step 5: train more epochs

In [ ]:
model = RNNModel(nn.LSTM, 2, 80, 3, False, 16).to(device)

In [ ]:
lr = 0.001
wd = 1e-5
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0

In [ ]:
num_epochs = 300

for epoch in range(num_epochs):
  # Train
  model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)
  # Validation
  loss_valid, acc_valid = validation(model,
                                     test_loader,
                                     loss_fn)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  acc_train_hist.append(acc_train)
  acc_valid_hist.append(acc_valid)

  print(f'Valid: Loss = {loss_valid:.4}, Acc = {acc_valid:.4}')

  if loss_valid < best_loss_valid:
    torch.save(model, 'model.pt')
    best_loss_valid = loss_valid
    print('Model Saved!')

  print()
  epoch_counter += 1

## **Plot**

In [ ]:
plt.plot(range(epoch_counter), loss_train_hist, 'r-', label='Train')
plt.plot(range(epoch_counter), loss_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('loss')
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(range(epoch_counter), acc_train_hist, 'r-', label='Train')
plt.plot(range(epoch_counter), acc_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()